# This notebook can be used to query NIC for fast ice data
The goal is to use the functions herewith to query for fast ice data, and optionally to append the results to the existing version of the covariate grid.

In [1]:
## Loading the libraries
list.of.packages <- c("rgdal", "proj4","rgeos","maptools","raster","stringr","plyr","dplyr","xml2","httr","ggplot2","data.table")
lapply(list.of.packages, require, character.only = TRUE)


Loading required package: rgdal

Loading required package: sp

rgdal: version: 1.2-20, (SVN revision 725)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.4-1 

Loading required package: proj4


Attaching package: ‘proj4’


The following object is masked from ‘package:rgdal’:

    project


Loading required package: rgeos

rgeos version: 0.5-2, (SVN revision 621)
 GEOS runtime version: 3.6.2-CAPI-1.10.2 
 Linking to sp version: 1.4-1 
 Polygon checking: TRUE 


Loading required package: maptools

Checking rgeos availability: TRUE

Loading required package: raster

Loading required package: stringr

Loading required package: plyr

Loading required package: dplyr


Attaching package: ‘dplyr

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

In [2]:
## Setting needed directories
pathToGit<-"/home/ubuntu/fastIce/FastIceCovars/"
nicsavedir<-"/home/ubuntu/fastIce/nictemp/"
nicresultsdir<-"/home/ubuntu/fastIce/nicresults/"


In [54]:
## Sourcing the utilities file
source(paste0(pathToGit,"scripts/fastIceCovars_utils.R"))
nicDownload<-function(filename,nicsavedir){
	Filelocation<-paste0("https://www.natice.noaa.gov/pub/weekly/antarctic/",filename[1])
	savename<-paste0(nicsavedir,"/",substr(filename[1],29,46))	
	download.file(Filelocation,destfile=savename,method="libcurl")
	unzip(zipfile=savename,exdir=nicsavedir)
	return(savename)
}
nicDownload<-function(filename,nicsavedir){
	Filelocation<-paste0("https://www.natice.noaa.gov/pub/weekly/antarctic/",filename[1])
	savename<-paste0(nicsavedir,substr(filename[1],29,46))	
	download.file(Filelocation,destfile=savename,method="libcurl")
	unzip(zipfile=savename,exdir=paste0(nicsavedir,substr(filename[1],29,40)))
	return(savename)
}

getFastIce<-function(fileloc,dataproj,nicsavedir){
	print("processing single fast ice date")
	
	#Read shapefile
	loopdsn <-substr(fileloc,1,nchar(fileloc)-4)
	dirnam<-str_sub(fileloc,-16,-5)
	shplst<-list.files(paste0(nicsavedir,dirnam),pattern=".shp")
	shpname<-substr(shplst[1],1,regexpr(".shp",shplst[1])-1)
		
	#read shapefile
	region <- readOGR(dsn=loopdsn,shpname)
	
	#Project to match full points shapefile
	region<-spTransform(region,dataproj)
	
	# Define fast ice and get region layer for creating land and ocean edges
	region@data$FP <- ifelse(region@data$FP == "08",8,0)
	#get fast ice only regions
	fast<-subset(region,region@data$FP == 8)
	
	functionList <- list("fast" = fast, "region" = region)
	
	return(functionList)
	
}


In [5]:
## Load the spatialpoints data.frame
load(paste0(pathToGit,"data/studyarea_points_wNearLand.RData"))
head(studyarea_pointswLand)

,pointid,meanslope,meanbathy,slope,bathy,shoredist,glacierdist,nearLineId,near_x,near_y,distToShore,adpedist,adpecol,empedist,empecol,cont300dist,cont800dist
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>
1,68791,2.621992,-2846.908,2.5563672,-2864.911,206728.3,247849.6,2351,-2273383,2327082,201526.4,207815.2,Ferrier Peninsula,1013124,Snow Hill Island,142791.6,588996.1
2,68792,4.591249,-2587.330,4.4836812,-2612.724,208790.2,250334.0,2351,-2273383,2327082,203621.5,209900.1,Ferrier Peninsula,1014024,Snow Hill Island,142791.6,588996.1
3,68819,1.411659,-3014.080,1.5170590,-3024.577,200154.1,241050.2,2351,-2273383,2327082,194935.5,201228.9,Ferrier Peninsula,1007323,Snow Hill Island,138016.9,584654.8
4,68820,1.413779,-2920.630,0.7565953,-2930.036,202161.9,243501.3,2351,-2273383,2327082,196973.9,203258.3,Ferrier Peninsula,1008204,Snow Hill Island,138016.9,584654.8
5,68821,4.561732,-2686.220,4.5623317,-2716.875,204263.8,246029.6,2351,-2273383,2327082,199116.9,205389.5,Ferrier Peninsula,1009109,Snow Hill Island,138016.9,584654.8
6,68822,4.902608,-2264.930,5.1146288,-2277.257,206460.4,248632.8,2351,-2273383,2327082,201361.3,207619.2,Ferrier Peninsula,1010037,Snow Hill Island,138016.9,584654.8


In [59]:
## Attribute with ice stability, persistence, and predictability
spdf<-studyarea_pointswLand
iceareas<-""
iceyear<-2011



[1] "processing single fast ice date"
OGR data source with driver: ESRI Shapefile 
Source: "/home/ubuntu/fastIce/nictemp/antarc111226", layer: "ANTARC111226"
with 393 features
It has 18 fields


,pointId,DecemberIcePresence
,<fct>,<dbl>
1,1021873,0
2,1021874,0
3,1021875,0
4,1021876,0
5,1021877,0
6,1021878,0
